In [1]:
from bayesbridge.reg_coef_sampler.hamiltonian_bps.hbps_dynamics import HBPSDynamics
from bayesbridge.reg_coef_sampler.hamiltonian_bps.hbps import HBPSSampler
from bayesbridge.reg_coef_sampler.hamiltonian_bps.dynamics import move_position, reflect_velocity
from mcmc_diagnostics import estimate_ess
import numpy as np

Trivariate normal with $x_1 \perp \!\!\!\!\! \perp x_3 \mid x_2$.

In [2]:
rho12 = rho23 = 0.5
rho13 = rho12 * rho23
cov = np.array([[1, rho12, rho13], 
                [rho12, 1, rho23],
                [rho13, rho23, 1]])
prec = np.linalg.inv(cov)
denom = 2*(rho12**2 + rho13**2 + rho23**2 - 2 * rho12 * rho13 * rho23 - 1)
d = cov.shape[1]

def f(x, loglik_only=False):
    """Full likelihood."""
    loglik = -0.5 * x @ prec  @ x
    if loglik_only:
        return loglik, None
    return loglik, -prec @ x

def f1(x, loglik_only=False):
    """(x_1, x_2) factor likelihood."""
    x1 = x[0]
    x2 = x[1]
    loglik = -(x1**2*(rho23**2-1)+x2**2*(rho13**2-1)/2 +2*x1*x2*(rho12-rho13*rho23))/denom
    if loglik_only:
        return  loglik, None
    return loglik, -np.array([2*x1*(rho23**2-1)+2*x2*(rho12-rho13*rho23), 2*x2*(rho13**2-1)/2 +2*x1*(rho12-rho13*rho23)])/denom

def f2(x, loglik_only=False):
    """(x_2, x_3) factor likelihood."""
    x1 = x[0]
    x2 = x[1]
    loglik = -(x1**2*(rho13**2-1)/2 + x2**2*(rho12**2-1)+2*x1*x2*(rho23-rho12*rho13))/denom
    if loglik_only:
        return  loglik, None
    return loglik, -np.array([2*x1*(rho13**2-1)/2 +2*x2*(rho23-rho12*rho13), 2*x2*(rho12**2-1)+2*x1*(rho23-rho12*rho13)])/denom


In [3]:
x = np.array([-0.5, 0.6, 0.7])

In [4]:
grad = np.zeros(3)
grad[:2] += f1(x[:2])[1] 
grad[-2:] += f2(x[-2:])[1]
f1(x[:2])[0] + f2(x[-2:])[0], grad

(-0.7133333333333332, array([ 1.06666667, -0.86666667, -0.53333333]))

In [5]:
f(x)

(-0.7133333333333333, array([ 1.06666667, -0.86666667, -0.53333333]))

## Standard HBPS

In [6]:
n = 10000
samples = np.empty((d, n))

In [7]:
%%time
np.random.seed(0)
# arbitrary initial position and integration time
x = np.array([-0.5, 0.6, 0.7])  
t = 2
for i in range(n):
    v = np.random.normal(size=3)
    inertia = np.random.exponential(1)
    x = HBPSSampler.generate_next_state(f, x, v, inertia, t)[0]
    samples[:, i] = x

CPU times: user 8.38 s, sys: 47.1 ms, total: 8.43 s
Wall time: 8.33 s


In [8]:
np.mean(samples, axis=1), np.cov(samples)

(array([0.00287305, 0.00444365, 0.01142087]),
 array([[0.98722259, 0.48479526, 0.2460593 ],
        [0.48479526, 0.99135137, 0.5043487 ],
        [0.2460593 , 0.5043487 , 1.0091644 ]]))

In [9]:
estimate_ess(samples.T)

array([6143.6597384 , 5008.40787224, 5407.26120169])

## Local HBPS

In [10]:
def next_bounce_time(fs, x, v, inertias, factors):
    bounce_times = []
    logps = []
    for f, inertia, factor in zip(fs, inertias, factors):
        logp = f(x[factor], loglik_only=True)[0]
        bounce_time = HBPSDynamics.next_bounce_time(f, None, x[factor], v[factor], inertia, -logp)
        bounce_times.append(bounce_time)
        logps.append(logp)
    next_bounce = min(bounce_times)
    bounce_factor = np.argmin(bounce_times)
    assert np.isfinite(next_bounce)
    return next_bounce, bounce_factor, logps

def update_inertias(x, inertias, old_logps, fs, factors, bounce_factor):
    for i, f in enumerate(fs):
        if i == bounce_factor:
            inertias[i] = 0
        else:
            new_logp, _ = fs[i](x[factors[i]], loglik_only=True)
            inertias[i] = inertias[i]- old_logps[i] + new_logp
    return inertias

def generate_next_state_local(fs, x, v, inertias, t, factors):
    total_time = 0
    while total_time < t:
        remaining_time = t-total_time
        next_bounce, bounce_factor, logps = next_bounce_time(fs, x, v, inertias, factors)
        if remaining_time < next_bounce:
            return move_position(x, v, remaining_time)
        else:
            x = move_position(x, v, next_bounce)
            _, grad = fs[bounce_factor](x[factors[bounce_factor]])
            grad = -grad
            v[factors[bounce_factor]] = reflect_velocity(v[factors[bounce_factor]], grad)
            inertias = update_inertias(x, inertias, logps, fs, factors, bounce_factor)
            total_time += next_bounce


### Single factor (non-local) using local code.

In [11]:
n = 10000
samples = np.empty((d, n))

In [12]:
%%time
np.random.seed(0)
x = np.array([-0.5, 0.6, 0.7])
t = 2
fs = [f]
factors = [slice(0,3)]
for i in range(n):
    v = np.random.normal(size=3)
    inertias = np.random.exponential(size=len(fs))
    x = generate_next_state_local(fs, x, v, inertias, t, factors)
    samples[:, i] = x

CPU times: user 11.5 s, sys: 15.7 ms, total: 11.5 s
Wall time: 11.5 s


In [13]:
np.mean(samples, axis=1), np.cov(samples)

(array([0.00089618, 0.00428629, 0.00114301]),
 array([[1.02082372, 0.50035752, 0.25806031],
        [0.50035752, 1.00997449, 0.50939196],
        [0.25806031, 0.50939196, 1.02975557]]))

In [14]:
estimate_ess(samples.T)

array([5925.74438395, 4988.78881791, 5774.02659024])

### Two factors

In [15]:
n = 10000
samples = np.empty((d, n))

In [16]:
%%time
np.random.seed(0)
x = np.array([-0.5, 0.6, 0.7])
t = 2
fs = [f1, f2]
factors = [slice(0,2), slice(1,3)]
for i in range(n):
    v = np.random.normal(size=3)
    inertias = np.random.exponential(size=len(fs))
    x = generate_next_state_local(fs, x, v, inertias, t, factors)
    samples[:, i] = x

CPU times: user 27.7 s, sys: 22.8 ms, total: 27.7 s
Wall time: 27.7 s


In [17]:
np.mean(samples, axis=1), np.cov(samples)

(array([7.75482031e-04, 1.32057931e-02, 9.55090005e-05]),
 array([[0.96949429, 0.48203708, 0.24816796],
        [0.48203708, 0.98258567, 0.48981733],
        [0.24816796, 0.48981733, 0.98257495]]))

In [18]:
estimate_ess(samples.T)

array([5310.32370348, 5676.42813025, 5470.24124804])